# Análisis de Métricas de Trámites Online

Este notebook analiza las métricas de trámites online y recursos de infraestructura (CPU, RAM, storage, input/output) cada 30 minutos.

In [ ]:
import os
import pandas as pd

csv_path = 'metricas_tramites_online.csv'
if not os.path.exists(csv_path):
    import subprocess
    subprocess.run(['python', 'generar_metricas_online.py'], check=True)

df = pd.read_csv(csv_path)
df.head()

In [ ]:
import json

with open('limites_metricas_online.json') as f:
    limites = json.load(f)
limites

## Visualización básica de las métricas

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(4, 1, figsize=(14, 16), sharex=True)

axs[0].plot(df['fecha_hora'], df['tramites_online'], color='blue')
axs[0].set_title('Trámites Online')
axs[0].set_ylabel('Cantidad')

axs[1].plot(df['fecha_hora'], df['cpu_usage'], color='red', label='CPU Usage')
axs[1].axhline(limites['cpu_usage'], color='black', linestyle='dashed', label='Límite CPU')
axs[1].set_title('CPU Usage (%)')
axs[1].set_ylabel('%')
axs[1].legend()

axs[2].plot(df['fecha_hora'], df['ram_usage'], color='green', label='RAM Usage')
axs[2].axhline(limites['ram_usage'], color='black', linestyle='dashed', label='Límite RAM')
axs[2].set_title('RAM Usage (GB)')
axs[2].set_ylabel('GB')
axs[2].legend()

axs[3].plot(df['fecha_hora'], df['storage_usage'], color='purple', label='Storage Usage')
axs[3].axhline(limites['storage_usage'], color='black', linestyle='dashed', label='Límite Storage')
axs[3].set_title('Storage Usage (GB)')
axs[3].set_ylabel('GB')
axs[3].set_xlabel('Fecha y Hora')
axs[3].legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Análisis automático de los datos usando un LLM
import requests

ENDPOINT = "<ENDPOINT>"
MODEL_NAME = "<MODEL_NAME>"
API_KEY = "<API_KEY>"

# Resumir los datos para enviar al LLM
resumen = df.describe().to_string()
prompt = f"Analiza las siguientes métricas de trámites online y recursos cada 30 minutos. Entrega insights, anomalías y recomendaciones para optimización.\n\n{resumen}" 

def obtener_analisis_llm(prompt):
    url = f"{ENDPOINT}/completions"
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "max_tokens": 400
    }
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()["choices"][0]["text"]

analisis_llm = obtener_analisis_llm(prompt)
print(analisis_llm)


In [ ]:
# Predicción de trámites online para los próximos x días usando Prophet
from prophet import Prophet

# Configura el número de días a predecir
dias_prediccion = 7  # Cambia este valor según lo que necesites

# Prepara los datos para Prophet
df_pred = df[['fecha_hora', 'tramites_online']].copy()
df_pred.rename(columns={'fecha_hora': 'ds', 'tramites_online': 'y'}, inplace=True)
df_pred['ds'] = pd.to_datetime(df_pred['ds'])

# Entrena el modelo
modelo = Prophet()
modelo.fit(df_pred)

# Genera fechas futuras
future = modelo.make_future_dataframe(periods=dias_prediccion*48, freq='30min')
prediccion = modelo.predict(future)

# Visualiza la predicción
plt.figure(figsize=(14,6))
plt.plot(df_pred['ds'], df_pred['y'], label='Histórico')
plt.plot(prediccion['ds'], prediccion['yhat'], label='Predicción', linestyle='dashed')
plt.title(f'Predicción de trámites online para los próximos {dias_prediccion} días')
plt.xlabel('Fecha y Hora')
plt.ylabel('Trámites Online')
plt.legend()
plt.tight_layout()
plt.show()

# Muestra la tabla de predicción para los próximos x días
prediccion[['ds', 'yhat']].tail(dias_prediccion*48)

In [ ]:
# Predicción y gráficos para cada métrica usando Prophet
from prophet import Prophet

metricas = ['tramites_online', 'cpu_usage', 'ram_usage', 'storage_usage', 'input_bytes', 'output_bytes']
dias_prediccion = 7  # Cambia este valor según lo que necesites

for metrica in metricas:
    print(f"\n--- Predicción para: {metrica} ---")
    df_pred = df[['fecha_hora', metrica]].copy()
    df_pred.rename(columns={'fecha_hora': 'ds', metrica: 'y'}, inplace=True)
    df_pred['ds'] = pd.to_datetime(df_pred['ds'])
    modelo = Prophet()
    modelo.fit(df_pred)
    future = modelo.make_future_dataframe(periods=dias_prediccion*48, freq='30min')
    prediccion = modelo.predict(future)
    plt.figure(figsize=(14,6))
    plt.plot(df_pred['ds'], df_pred['y'], label='Histórico')
    plt.plot(prediccion['ds'], prediccion['yhat'], label='Predicción', linestyle='dashed')
    plt.title(f'Predicción de {metrica} para los próximos {dias_prediccion} días')
    plt.xlabel('Fecha y Hora')
    plt.ylabel(metrica)
    plt.legend()
    plt.tight_layout()
    plt.show()
    # Muestra la tabla de predicción para los próximos x días
    display(prediccion[['ds', 'yhat']].tail(dias_prediccion*48))

In [ ]:
# Análisis automático de las predicciones usando LLM
import requests

ENDPOINT = "<ENDPOINT>"
MODEL_NAME = "<MODEL_NAME>"
API_KEY = "<API_KEY>"

metricas = ['tramites_online', 'cpu_usage', 'ram_usage', 'storage_usage', 'input_bytes', 'output_bytes']
dias_prediccion = 7  # Cambia este valor según lo que necesites

for metrica in metricas:
    print(f"\n--- Análisis LLM para: {metrica} ---")
    df_pred = df[['fecha_hora', metrica]].copy()
    df_pred.rename(columns={'fecha_hora': 'ds', metrica: 'y'}, inplace=True)
    df_pred['ds'] = pd.to_datetime(df_pred['ds'])
    modelo = Prophet()
    modelo.fit(df_pred)
    future = modelo.make_future_dataframe(periods=dias_prediccion*48, freq='30min')
    prediccion = modelo.predict(future)
    resumen_pred = prediccion[['ds', 'yhat']].tail(dias_prediccion*48).describe().to_string()
    prompt = f"Analiza la predicción de la métrica '{metrica}' para los próximos {dias_prediccion} días. Entrega insights, anomalías y recomendaciones.\n\n{resumen_pred}"
    def obtener_analisis_llm(prompt):
        url = f"{ENDPOINT}/completions"
        payload = {
            "model": MODEL_NAME,
            "prompt": prompt,
            "max_tokens": 400
        }
        headers = {
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        }
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["text"]
    analisis_llm = obtener_analisis_llm(prompt)
    print(analisis_llm)
